In [24]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import itertools

In [25]:
def I12(x, y):
    return -1*(x**2) + 12*x + 0.3 - 5 + y**2 - 10*y

def I21(x, y):
    return x**2 + 3*y - 5*x + 0.1*x*y - 1.5*(y**2) + 30

In [26]:
step1, step2 = 1, 1
a1, b1 = 0, 5
a2, b2 = 0, 5

In [27]:
x_grid = [np.arange(a1, b1+step1/10, step1), np.arange(a2, b2+step2/10, step2)]
indexes = list(itertools.product(x_grid[0], x_grid[1]))
index = pd.MultiIndex.from_tuples(indexes, names=["x", "y"])
values = np.array([(I12(x, y), I21(x, y)) for x in x_grid[0] for y in x_grid[1]])

In [28]:
df = pd.DataFrame(values, index=index, columns=["I12", "I21"])
df

I12   I21
x   y              
0.0 0.0  -4.7  30.0
    1.0 -13.7  31.5
    2.0 -20.7  30.0
    3.0 -25.7  25.5
    4.0 -28.7  18.0
    5.0 -29.7   7.5
1.0 0.0   6.3  26.0
    1.0  -2.7  27.6
    2.0  -9.7  26.2
    3.0 -14.7  21.8
    4.0 -17.7  14.4
    5.0 -18.7   4.0
2.0 0.0  15.3  24.0
    1.0   6.3  25.7
    2.0  -0.7  24.4
    3.0  -5.7  20.1
    4.0  -8.7  12.8
    5.0  -9.7   2.5
3.0 0.0  22.3  24.0
    1.0  13.3  25.8
    2.0   6.3  24.6
    3.0   1.3  20.4
    4.0  -1.7  13.2
    5.0  -2.7   3.0
4.0 0.0  27.3  26.0
    1.0  18.3  27.9
    2.0  11.3  26.8
    3.0   6.3  22.7
    4.0   3.3  15.6
    5.0   2.3   5.5
5.0 0.0  30.3  30.0
    1.0  21.3  32.0
    2.0  14.3  31.0
    3.0   9.3  27.0
    4.0   6.3  20.0
    5.0   5.3  10.0

In [29]:
newValues = np.reshape(values[:,0], (6,6)).T
df1 = pd.DataFrame(newValues)
df1

,0,1,2,3,4,5
0,-4.7,6.3,15.3,22.3,27.3,30.3
1,-13.7,-2.7,6.3,13.3,18.3,21.3
2,-20.7,-9.7,-0.7,6.3,11.3,14.3
3,-25.7,-14.7,-5.7,1.3,6.3,9.3
4,-28.7,-17.7,-8.7,-1.7,3.3,6.3
5,-29.7,-18.7,-9.7,-2.7,2.3,5.3


In [30]:
newValues = np.reshape(values[:,1], (6,6)).T
df1 = pd.DataFrame(newValues)
df1

,0,1,2,3,4,5
0,30.0,26.0,24.0,24.0,26.0,30.0
1,31.5,27.6,25.7,25.8,27.9,32.0
2,30.0,26.2,24.4,24.6,26.8,31.0
3,25.5,21.8,20.1,20.4,22.7,27.0
4,18.0,14.4,12.8,13.2,15.6,20.0
5,7.5,4.0,2.5,3.0,5.5,10.0


In [31]:
max_x_min_y = df.groupby("x").min().max()["I12"]
print("I12*")
I12_s = df.loc[df['I12'] == max_x_min_y]["I12"]
print(I12_s)
max_y_min_x = df.groupby("y").min().max()["I21"]
print("\nI21*")
I21_s = df.loc[df['I21'] == max_y_min_x]["I21"]
print(I21_s)

I12*
x    y  
5.0  5.0    5.3
Name: I12, dtype: float64

I21*
x    y  
2.0  1.0    25.7
Name: I21, dtype: float64


In [32]:
R1 = np.array([[0.1,  0.2,  0.25, 0.4,  0.4,  0.4], # ns
               [0.05, 0.05, 0.05, 0.1,  0.05, 0.05], # fm
               [0.1,  0.2,  0.3,  0.25, 0.25, 0.15]])# in

R2 = np.array([[0.15, 0.02, 0.3,  0.3,  0.2,  0.1], # ns
               [0.05, 0.1,  0.15, 0.15, 0.25, 0.2],  # fm
               [0.3,  0.25, 0.3,  0.4,  0.5,  0.35]]) # in

In [33]:
def J_12_ns(x, y):
    return 0.1*x * (0.1*y + 2)

def J_12_fm(x, y):
    return (-0.2*y + 0.05*x*y + 2*x + 2)/3

def J_12_in(x, y):
    return (y - 0.1*x*y + 2*x + 20)/10

In [34]:
def J_21_ns(x, y):
    return 0.3*y * (0.1*x + 2.5)

def J_21_fm(x, y):
    return (y + 0.05*x*y + 0.25*x + 3)/2

def J_21_in(x, y):
    return (y - 0.1*x*y + 0.25 + x)/3

In [35]:
def calc_f_sum_12():
    x = int(I12_s.index.codes [0][0])
    y = int(I12_s.index.codes [1][0])
    val = I12_s.values[0]
    
    _ns, _fm, _in = R1[:, y]
    
    return (1- _ns)*(1-_fm)*(1-_in)*val - (_ns*J_12_ns(x, y) + _fm*J_12_fm(x, y) + _in*J_12_in(x, y))

In [36]:
def calc_f_sum_21():
    x = int(I21_s.index.codes [0][0])
    y = int(I21_s.index.codes [1][0])
    val = I21_s.values[0]

    _ns, _fm, _in = R2[:, x]
    
    return (1- _ns)*(1-_fm)*(1-_in)*val - (_ns*J_21_ns(x, y) + _fm*J_21_fm(x, y) + _in*J_21_in(x, y))

In [37]:
f_sum_12 = calc_f_sum_12()
f_sum_21 = calc_f_sum_21()
print(f"F_sum_12 = {f_sum_12}")
print(f"F_sum_21 = {f_sum_21}")

F_sum_12 = 1.3761833333333315
F_sum_21 = 9.811049999999998


In [38]:
def calc_unfavorable_first():
    p = np.zeros(R1.shape[1])
    for i in range(len(p)):
        _mul = 1
        for j in range(R1.shape[0]):
            _mul = _mul * R1[j, i]
        p[i] = _mul
    
    y_idx = np.argmax(p)
    p_max = max(p)
    _df = df.query(f"y == {y_idx}")["I12"]
    max_val = _df.max()
    x_idx = _df.idxmax()[0]
    
    return (x_idx, y_idx, max_val)

In [39]:
def calc_unfavorable_second():
    p = np.zeros(R2.shape[1])
    for i in range(len(p)):
        _mul = 1
        for j in range(R2.shape[0]):
            _mul = _mul * R1[j, i]
        p[i] = _mul
    
    x_idx = np.argmax(p)
    p_max = max(p)
    _df = df.query(f"x == {x_idx}")["I21"]
    max_val = _df.max()
    y_idx = _df.idxmax()[0]
    
    return (x_idx, y_idx, max_val)

In [40]:
unfavorable_first = calc_unfavorable_first()
unfavorable_second = calc_unfavorable_second()
print(unfavorable_first)
print(unfavorable_second)

(5.0, 3, 9.299999999999997)
(3, 3.0, 25.8)


In [41]:
def calc_unfavorable_f_sum_12(x, y, val):    
    _ns, _fm, _in = R1[:, y]
    return (1- _ns)*(1-_fm)*(1-_in)*val - (_ns*J_12_ns(x, y) + _fm*J_12_fm(x, y) + _in*J_12_in(x, y))

In [42]:
def calc_unfavorable_f_sum_21(x, y, val):
    _ns, _fm, _in = R2[:, x]  
    return (1- _ns)*(1-_fm)*(1-_in)*val - (_ns*J_21_ns(x, y) + _fm*J_21_fm(x, y) + _in*J_21_in(x, y))

In [43]:
unfavorable_f_sum_12 = calc_unfavorable_f_sum_12(*unfavorable_first)
unfavorable_f_sum_21 = calc_unfavorable_f_sum_21(*unfavorable_second)
print(f"F'_sum_12 = {unfavorable_f_sum_12}")
print(f"F'_sum_21 = {unfavorable_f_sum_21}")

F'_sum_12 = 2.1139999999999994
F'_sum_21 = 7.201266666666666


In [3]:
# pip install pandas --upgrade

In [4]:
# pip install iexfinance --upgrade